In [ ]:
import pandas as pd
import collections

In [ ]:
#https://dados.educacao.sp.gov.br/search/field_topic/desafios-seduc-164

# FATO
- Avaliacao do Saresp
- Ano
- Valor (medprof)
- Dim disciplina (serie/periodo/conteudo) - TODO: incluir o professor aqui.
- Dim equipe de gestores - tabela com Ids e uma auxiliar de detalhamentos
- Dim escola

# FUNCOES

In [273]:
def limpa_serie_ano(serie_ano):
    if serie_ano == '9Âº Ano EF' or serie_ano == '9º Ano EF':
        return 'EF-9'
    elif serie_ano == '7Âº Ano EF':
        return 'EF-7'
    elif serie_ano == 'EM-3Âª sÃ©rie' or serie_ano == 'EM-3ª série':
        return 'EM-3'
    elif serie_ano == '3Âº Ano EF' or serie_ano == '3º Ano EF':
        return 'EF-3'
    elif serie_ano == '5Âº Ano EF' or serie_ano == '5º Ano EF':
        return 'EF-5'
    
def limpa_ds_comp(ds_comp):
    if ds_comp == 'LÃ\x8dNGUA PORTUGUESA' or ds_comp == 'LÍNGUA PORTUGUESA':
        return 'PORTUGUES'
    elif ds_comp == 'MATEMÃ\x81TICA' or ds_comp == 'MATEMÁTICA':
        return 'MATEMATICA'
    elif ds_comp == 'CIÃ\x8aNCIAS':
        return 'CIENCIAS'    
    
def enulo(valor):
    return str(valor) == 'nan'   

# CARREGAMENTO DOS DADOS

In [ ]:
# SARESP
arquivo = 'dados/saresp/SARESP_escolas_2014.csv'
df_2014 = pd.read_csv(arquivo, sep=';', encoding='latin-1')
df_2014['ano'] = 2014
arquivo = 'dados/saresp/SARESP_escolas_2015.csv'
df_2015 = pd.read_csv(arquivo, sep=';', encoding='latin-1')
df_2015['ano'] = 2015
arquivo = 'dados/saresp/SARESP_escolas_2016.csv'
df_2016 = pd.read_csv(arquivo, sep=';', encoding='latin-1')
df_2016['ano'] = 2016
arquivo = 'dados/saresp/SARESP_escolas_2017.csv'
df_2017 = pd.read_csv(arquivo, sep=';', encoding='latin-1')
df_2017['ano'] = 2017
arquivo = 'dados/saresp/SARESP_escolas_2018.csv'
df_2018 = pd.read_csv(arquivo, sep=';', encoding='latin-1')
df_2018['ano'] = 2018
saresp = df_2014.append(df_2015).append(df_2016).append(df_2017).append(df_2018)
saresp = saresp[saresp.medprof.notnull()] # nao estou interessado se nao tiver o valor do saresp

In [ ]:
# GESTORES
arquivo = 'dados\lideres\lideres_DIRETORES DE ESCOLA.csv'
diretor = pd.read_csv(arquivo, sep=';')
arquivo = 'dados\lideres\lideres_VICE_DIRETOR.csv'
vice = pd.read_csv(arquivo, sep=';')
arquivo = 'dados\lideres\lideres_PROFESSOR_COORDENADOR.csv'
coordenador = pd.read_csv(arquivo, sep=';')
vice = vice.drop('Unnamed: 16',axis=1)

In [ ]:
# IDESP
df_idesp_anosiniciais = pd.read_csv('dados\idesp/IDESP_Escolas_2007_2018_AI.CSV',sep=',')
df_idesp_anosfinais = pd.read_csv('dados\idesp/IDESP_Escolas_2007_2018_AF.CSV',sep=',')
df_idesp_ensinomedio = pd.read_csv('dados\idesp/IDESP_Escolas_2007_2018_EM.CSV',sep=',')

In [ ]:
df_idesp = df_idesp_anosiniciais.append(df_idesp_anosfinais).append(df_idesp_ensinomedio)

# GERANDO DIMENSAO IDESP

In [442]:
CRIMPAR_MAX_VARIACAO = 30 # 90%
CRIMPAR_MIN_VARIACAO = -13 # 10%

In [278]:
LISTA_ANOS = ['2014', '2015', '2016', '2017', '2018']
LISTA_ANO_ANTERIOR = ['2013', '2014', '2015', '2016', '2017']
idesp = {}
idesp['id_gestao'] = []
idesp['id_escola'] = []
idesp['ano_gestao'] = []
idesp['nivel_ensino'] = []
idesp['nota_ano'] = []
idesp['nota_anterior'] = []

for _, row in df_idesp.iterrows():
    for ano,ano_anterior in zip(LISTA_ANOS, LISTA_ANO_ANTERIOR):
        if enulo(row.loc[ano]):
            continue
        idesp_anterior = row.loc[ano_anterior]
        if enulo(idesp_anterior):
            idesp_anterior = row.loc[ano]
            
        idesp['id_gestao'].append('esc-' + str(row.loc['CODIGO CIE']) + '-ano-' + str(ano))
        idesp['id_escola'].append(row.loc['CODIGO CIE'])
        idesp['ano_gestao'].append(ano)
        idesp['nivel_ensino'].append(row.loc['NIVEL ENSINO'])
        idesp['nota_ano'].append(row.loc[ano])
        idesp['nota_anterior'].append(idesp_anterior)

idesp = pd.DataFrame(data=idesp)

In [437]:
idesp_group = {}
idesp_group['id_gestao'] = []
idesp_group['id_escola'] = []
idesp_group['ano_gestao'] = []
idesp_group['nota_ano'] = []
idesp_group['variacao'] = []

for nome, dados in idesp.groupby('id_gestao'):
    nota_ano = dados.nota_ano.mean()
    nota_anterior = dados.nota_anterior.mean()
    variacao = int(100 * (nota_ano - nota_anterior) / nota_anterior)
    idesp_group['id_gestao'].append(nome)
    idesp_group['id_escola'].append(dados.id_escola.iloc[0])
    idesp_group['ano_gestao'].append(int(dados.ano_gestao.iloc[0]))
    idesp_group['nota_ano'].append(nota_ano)
    idesp_group['variacao'].append(variacao)
    
idesp_group = pd.DataFrame(data=idesp_group)

In [443]:
idesp_group.variacao = idesp_group.variacao.apply(lambda x: CRIMPAR_MAX_VARIACAO if x > CRIMPAR_MAX_VARIACAO else x)
idesp_group.variacao = idesp_group.variacao.apply(lambda x: CRIMPAR_MIN_VARIACAO if x < CRIMPAR_MIN_VARIACAO else x)

In [444]:
idesp_group.variacao.describe(percentiles=[0.10,0.90])

count    25111.000000
mean         6.341603
std         13.379996
min        -13.000000
10%        -13.000000
50%          4.000000
90%         30.000000
max         30.000000
Name: variacao, dtype: float64

### ESTUDO DAS VARIACOES

In [320]:
idesp_group[idesp_group.variacao > 200]

,id_gestao,id_escola,ano_gestao,nota_ano,variacao
1033,esc-12488-ano-2015,12488,2015,2.87,304
4102,esc-2008-ano-2016,2008,2016,2.46,261
7679,esc-29269-ano-2015,29269,2015,7.96,237
11831,esc-40551-ano-2016,40551,2016,2.25,324
12296,esc-4157-ano-2014,4157,2014,3.50,379
12667,esc-42500-ano-2018,42500,2018,3.70,345
12717,esc-42626-ano-2018,42626,2018,3.08,201


In [321]:
df_idesp[df_idesp.loc[:,'CODIGO CIE'] == 40551]

,CODIGO CIE,CODIGO INEP,DIRETORIA,ESCOLA,MUNICIPIO,NIVEL ENSINO,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
2253,40551,35040551.0,ITAPEVI,LENIO VIEIRA DE MORAES PROFESSOR ...,BARUERI,ENSINO MEDIO,0.76,1.14,0.96,0.86,1.3,1.15,1.27,1.0,0.53,2.25,1.77,1.56


# GERANDO DIMENSAO GESTORES
- cd_escola
- ano
- diretoria
- id_diretor - detalhamento aparece em outro lugar
- id_vice
- id_coordenador

### IMPORTANTE: A gestao 'esc-18314-ano-2018' teve 8 diretores, meio complicado de avaliar, ne.
- equipe completa? 
- num_mudancas de gestores
- atribuir numeros da proeficiencia como 1,2,3,4,5 na ordem de qualidade dos anos analisados.
- AVALIAR TEMPO COM A MESMA EQUIPE DE GESTORES

In [ ]:
LISTA_ANOS = ['2014', '2015', '2016', '2017', '2018']
def foi_gestor(ano):
    return ano == 'SIM'

In [ ]:
df_gestao = {}
df_gestao['id_gestao'] = []
df_gestao['id_escola'] = []
df_gestao['id_gestor'] = []
df_gestao['ano_gestao'] = []
df_gestao['diretoria'] = []
df_gestao['funcao'] = []

for _, row in diretor.iterrows():
    for ANO in LISTA_ANOS:
        if foi_gestor(row.loc[ANO]):
            df_gestao['id_gestao'].append('esc-' + str(row.CD_ESCOLA) + '-ano-' + str(ANO))
            df_gestao['id_escola'].append(row.CD_ESCOLA)
            df_gestao['id_gestor'].append(row.id_interno)
            df_gestao['ano_gestao'].append(ANO)
            df_gestao['diretoria'].append(row.DIRETORIA)
            df_gestao['funcao'].append('DIRETOR')

for _, row in vice.iterrows():
    for ANO in LISTA_ANOS:
        if foi_gestor(row.loc[ANO]):
            df_gestao['id_gestao'].append('esc-' + str(row.CD_ESCOLA) + '-ano-' + str(ANO))
            df_gestao['id_escola'].append(row.CD_ESCOLA)
            df_gestao['id_gestor'].append(row.id_interno)
            df_gestao['ano_gestao'].append(ANO)
            df_gestao['diretoria'].append(row.DIRETORIA)
            df_gestao['funcao'].append('VICE')
            
for _, row in coordenador.iterrows():
    for ANO in LISTA_ANOS:
        if foi_gestor(row.loc[ANO]):
            df_gestao['id_gestao'].append('esc-' + str(row.CD_ESCOLA) + '-ano-' + str(ANO))
            df_gestao['id_escola'].append(row.CD_ESCOLA)
            df_gestao['id_gestor'].append(row.id_interno)
            df_gestao['ano_gestao'].append(ANO)
            df_gestao['diretoria'].append(row.DIRETORIA)
            df_gestao['funcao'].append('COORDENADOR')
            
df_gestao = pd.DataFrame(df_gestao)

### VERIFICANDO QUANTIDADE DE TROCAS DE GESTORES

In [ ]:
k = 0
contagem_gestores = collections.Counter(df_gestao.id_gestao)
for id_gestao in contagem_gestores:
    if contagem_gestores[id_gestao] == 3:
        k += 1

In [323]:
df_gestao[df_gestao.id_gestao == 'esc-472505-ano-2014']

,id_gestao,id_escola,id_gestor,ano_gestao,diretoria,funcao
28754,esc-472505-ano-2014,472505,334929,2014,SUL 3,DIRETOR
95938,esc-472505-ano-2014,472505,222988,2014,SUL 3,VICE
140523,esc-472505-ano-2014,472505,399478,2014,SUL 3,COORDENADOR


# GESTAO + IDESP

In [390]:
cont_gestores = {}
cont_gestores['id_gestao'] = []
cont_gestores['id_escola'] = []
cont_gestores['n_gestores'] = []

for nome, dados in df_gestao.groupby('id_gestao'):
    n_gestores = len(dados)
    cont_gestores['id_gestao'].append(nome)
    cont_gestores['id_escola'].append(dados.id_escola.iloc[0])
    cont_gestores['n_gestores'].append(n_gestores)
    
cont_gestores = pd.DataFrame(data=cont_gestores)
cont_gestores_aux = {}
cont_gestores_aux['id_escola'] = []
cont_gestores_aux['mediana'] = []
for nome, dados in cont_gestores.groupby('id_escola'):
    cont_gestores_aux['id_escola'].append(dados.id_escola.iloc[0])
    cont_gestores_aux['mediana'].append(dados.n_gestores.median())

cont_gestores_aux = pd.DataFrame(data=cont_gestores_aux)
cont_gestores = pd.merge(cont_gestores,
                  cont_gestores_aux,
                  on='id_escola')
cont_gestores['var_gestores'] = result.n_gestores - result.mediana

result = pd.merge(idesp_group,
                  cont_gestores,
                  on='id_gestao')
result.loc[:,['variacao','var_gestores']].corr()

# DIFERENTE DO QUE EU PENSAVA, PARECE QUE ESSE NUMERO ALTO DE GESTORES
# MAIS OU MENOS NORMAL E NAO IMPACTA MUITO OS RESULTADOS

,variacao,var_gestores
variacao,1.000000,0.006077
var_gestores,0.006077,1.000000


# NOTAS DOS GESTORES
- id_gestor=131444 pode atuar em mais de uma escola? (foi 3 em 2018)

In [492]:
result = pd.merge(idesp_group,
                  df_gestao,
                  on='id_gestao')

In [493]:
result.head(5)

,id_gestao,id_escola_x,ano_gestao_x,nota_ano,variacao,id_escola_y,id_gestor,ano_gestao_y,diretoria,funcao
0,esc-10005-ano-2014,10005,2014,3.48,10,10005,326625,2014,ITAPECERICA DA SERRA,DIRETOR
1,esc-10005-ano-2014,10005,2014,3.48,10,10005,157667,2014,ITAPECERICA DA SERRA,VICE
2,esc-10005-ano-2014,10005,2014,3.48,10,10005,314895,2014,ITAPECERICA DA SERRA,VICE
3,esc-10005-ano-2014,10005,2014,3.48,10,10005,351780,2014,ITAPECERICA DA SERRA,COORDENADOR
4,esc-10005-ano-2014,10005,2014,3.48,10,10005,398436,2014,ITAPECERICA DA SERRA,COORDENADOR


In [494]:
nota_gestor = {}
nota_gestor['id_gestor'] = []
nota_gestor['nota_var'] = []
nota_gestor['n_atuacoes'] = []
for nome, dados in result.groupby('id_gestor'):
    nota_gestor['id_gestor'].append(nome)
    nota_gestor['nota_var'].append(dados.variacao.mean())
    nota_gestor['n_atuacoes'].append(len(dados))

nota_gestor = pd.DataFrame(data=nota_gestor)

In [495]:
nota_gestor.nota_var.describe()

count    50969.000000
mean         6.537152
std          8.153998
min        -13.000000
25%          2.000000
50%          6.166667
75%         10.500000
max         30.000000
Name: nota_var, dtype: float64

In [496]:
nota_gestor.n_atuacoes.describe()

count    50969.000000
mean         3.251310
std          1.807438
min          1.000000
25%          2.000000
50%          3.000000
75%          5.000000
max         22.000000
Name: n_atuacoes, dtype: float64

In [514]:
nota_gestor = nota_gestor[nota_gestor.n_atuacoes >= 2]

In [516]:
nota_gestor.head(5)

,id_gestor,nota_var,n_atuacoes
0,131346,-1.250000,4
1,131376,6.500000,2
2,131393,8.333333,6
3,131418,7.400000,5
4,131444,12.428571,7


In [498]:
result[result.id_gestor == 355299]

,id_gestao,id_escola_x,ano_gestao_x,nota_ano,variacao,id_escola_y,id_gestor,ano_gestao_y,diretoria,funcao
65554,esc-3554-ano-2015,3554,2015,3.38,0,3554,355299,2015,CENTRO,DIRETOR
65563,esc-3554-ano-2016,3554,2016,3.92,15,3554,355299,2016,CENTRO,DIRETOR
65574,esc-3554-ano-2017,3554,2017,2.75,-13,3554,355299,2017,CENTRO,DIRETOR
65584,esc-3554-ano-2018,3554,2018,3.89,30,3554,355299,2018,CENTRO,DIRETOR


In [503]:
dados

,id_gestao,id_escola_x,ano_gestao_x,nota_ano,variacao,id_escola_y,id_gestor,ano_gestao_y,diretoria,funcao
157468,esc-923771-ano-2016,923771,2016,2.350,-13,923771,SIM,2016,GUARULHOS NORTE,VICE
157479,esc-923771-ano-2017,923771,2017,2.945,25,923771,SIM,2017,GUARULHOS NORTE,VICE
157487,esc-923771-ano-2018,923771,2018,2.645,-10,923771,SIM,2018,GUARULHOS NORTE,VICE


# DIMENSAO SARESP
- Substituida pela dimensao IDESP

In [ ]:
saresp.SERIE_ANO = saresp.SERIE_ANO.apply(limpa_serie_ano)
saresp.ds_comp = saresp.ds_comp.apply(limpa_ds_comp)
saresp.medprof = saresp.medprof \
                .apply(lambda x: str(x).replace('.','').replace(',','.')) \
                .astype(float) 

df_saresp = {}
df_saresp['id_gestao'] = []
df_saresp['id_escola'] = []
df_saresp['ano_gestao'] = []
df_saresp['serie_ano'] = []
df_saresp['periodo'] = []
df_saresp['disciplina'] = []
df_saresp['medprofici'] = []

for _, row in saresp.iterrows():
    df_saresp['id_gestao'].append('esc-' + str(row.CODESC) + '-ano-' + str(row.ano))
    df_saresp['id_escola'].append(row.CODESC)
    df_saresp['ano_gestao'].append(row.ano)
    df_saresp['serie_ano'].append(row.SERIE_ANO)
    df_saresp['periodo'].append(row.periodo)
    df_saresp['disciplina'].append(row.ds_comp)
    df_saresp['medprofici'].append(row.medprof)

df_saresp = pd.DataFrame(data=df_saresp) 

saresp_medias = {}
saresp_medias['id_gestao'] = []
saresp_medias['media'] = []
saresp_medias['id_escola'] = []
for nome, dados in df_saresp.groupby('id_gestao'):
    saresp_medias['id_gestao'].append(nome)
    saresp_medias['media'].append(dados.medprofici.mean())
    saresp_medias['id_escola'].append(dados.id_escola.iloc[0])

saresp_medias = pd.DataFrame(data = saresp_medias)